<a href="https://colab.research.google.com/github/flari-gold/IU/blob/main/IU_RedditDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Package Installs mit pip
!pip install praw
!pip install HanTa

#Import Modules
from google.colab import userdata
import praw
import collections
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from HanTa import HanoverTagger
import string
from gensim import corpora
from gensim.models import LdaModel


#Paket zur Visualisierung der Ergebnisse
!pip install pyLDAvis

#Module zur Visualisierung der Ergebnisse
import pyLDAvis
import pyLDAvis.gensim
from wordcloud import WordCloud
import matplotlib.pyplot as pyplot


In [ ]:
#Parameter für die Anmeldung
#Daten der unter https://old.reddit.com/prefs/apps/ generierten Anwendung. Ausreichend für ReadOnly Zugriffe.
#Die Daten sind im Google Colab Secrets hinterlegt
CLIENT_ID=userdata.get('CLIENT_ID')
CLIENT_SECRET=userdata.get('CLIENT_SECRET')

#Für Write Zugriffe muss ebenfalls noch der eigene Account angegeben werden, kann in diesem Fall ignoriert werden
#USERNAME=userdata.get('USERNAME')
#PASSWORD=userdata.get('PASSWORD')

#Parameter für die Datenanalyse
SUBREDDIT='Python'
ANZAHL_POSTS=100
TOPICS=5
PASSES=10
COMMENTLIMIT=None

In [ ]:
#Anfrage an die API und Speicherung

#Initialisierung reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent="Mein Topic Modeling Script", #User Agent ist zwingend notwendig
    check_for_async=False #PRAW wirft eine Warnung welche die ASYNC API empfiehlt, kann ignoriert werden
)

#Counter für die Metriken
user_upvotes = collections.Counter()
user_contributionscounter = collections.Counter()

post_numberofcomments = {} #Counter wäre möglich, Dictionary tut es hier auch

all_textdata = []
#Anfragen an die Reddit API mit über request/http liefern die Daten im JSON Format. PRAW vereinfacht dies, in dem man die direkt die Attribute ansprechen kann
for submission in reddit.subreddit(SUBREDDIT).hot(limit=ANZAHL_POSTS):

    #Hochzählen der Counter
    if submission.author != None:
        user_contributionscounter[submission.author] += 1
        user_upvotes[submission.author] += submission.score

    post_numberofcomments[submission.id] = submission.num_comments

    post_textdata = [submission.title]#Alle Reddit Beiträge haben einen Titel. Der Titel wird ebenfalls im Topic Modeling wie ein normaler Kommentar behandelt
    post_textdata.append(submission.selftext)#Nicht alle Beträge haben einen Selftext. Beiträge die auf Links verweisen beispielsweise. Falls Selftext vorhanden ist, wird er im Topic Modeling behandelt, ansonsten ist der String leer

#Sobald ein Kommntar auf Reddit zu viele Antworten hat, werden die Antworten hinter einer Schaltfläche "more" verborgen und das gleiche gilt auch beim Zugriff auf die API.
#Das Auflösen der "more" stellt einen eigenen API Aufruf da und sorgt aufgrund der Zugriff Limits bei größeren Anfragen zu einer langen Laufzeit
    submission.comments.replace_more(limit=COMMENTLIMIT)

    for comment in submission.comments.list():

        #Hochzählen der Counter
        if comment.author != None:
            user_contributionscounter[comment.author] += 1
            user_upvotes[comment.author] += comment.score

        post_textdata.append(comment.body)

    all_textdata.append(post_textdata)



In [ ]:
#Text Bereinigung

#Stopwörter von NLTK
stop_words = stopwords.words('english')
stop_words.extend(['would', 'get', 'thats', 'im', 'ive', 'dont', 'didnt', 'cant', 'also'])#Stopwords enthält standardmäßig 'i' und 'am' aber weder 'i'm' noch 'im'

#Lemmatiserung mit HanoverTagger
tagger_en = HanoverTagger.HanoverTagger('morphmodel_en.pgz')

#Zu entfernende Satzzeichen
punctuation = string.punctuation
punctuation += '’–'

all_cleantext = []
for i in all_textdata:

    string_result = []
    for j in i:

        templist = []
        #Entfernung der Satzzeichen
        tempstring = j.translate(str.maketrans('', '', punctuation))

        #Umwandlung in Kleinbuchstaben
        for word in tempstring.lower().split():
            #Lemmatisierung
            templist.append(tagger_en.analyze(word)[0])
        #Filterung von Stopwörtern
        templist = [word for word in templist if word not in stop_words]
        tempstring = ' '.join(templist)

        string_result.append(tempstring)
    all_cleantext.append(string_result)

#wird später für die Aufbereitung der Ergebnisse genutzt, schreibt alles in einen String
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string)
end_string = ' '.join(templist)


In [ ]:
#Erstellung des LDA Models mit Hilfe von Gensim


#all_cleantext ist eine Liste, welche Listen beinhaltet, welche wiederum Strings beinhalten
#Jeder String repräsentiert einen Post/Kommentar
#Für die Verarbeitung muss aber in der Liste für jedes Wort ein eigener String stehen
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string.split())


allwords = corpora.Dictionary(templist)
allposts = [allwords.doc2bow(text) for text in templist]

lda_model = LdaModel(allposts, num_topics=TOPICS, id2word=allwords, passes=PASSES)





In [ ]:
#Ausgabe der Ergebnisse in Textform

#Ausgabe der Keywords pro Topic
print("Analyse des Subreddits: " + SUBREDDIT)

print("Gewollte Anzahl an abgefragten Posts: " + str(ANZAHL_POSTS))
print("Tatsächlich abgefragt: " + str(len(post_numberofcomments)))#Falls das Subreddit nicht genug Posts beinhaltet
print("Durchschnittliche Anzahl an Kommentaren pro Post: " + str(sum(post_numberofcomments.values())/len(post_numberofcomments)))
print("Kleinsten Anzahl an Kommentaren pro Post: " + str(min(post_numberofcomments.values())))
print("Größte Anzahl an Kommentaren pro Post: " + str(max(post_numberofcomments.values())))

print("\nAnzahl zu ermittelten Topics: " + str(TOPICS))
print("Keywords pro ermitteltem Topic")
for t,x in lda_model.print_topics():
    print("Topic {}: {}".format(t+1, x))

#Häufigste Wörter

print("\nHäufigste Wörter")
for t,x in collections.Counter(end_string.split()).most_common(10):
    print("{}: {}".format(t, x))

#Ausgabe der aktivsten User
print("\nMeiste Anzahl Beiträge pro User")
for t,x in user_contributionscounter.most_common(10):
    print("{}: {}".format(t, x))

#Ausgabe der User mit den meisten Upvotes
print("\nMeiste Anzahl Upvotes pro User")
for t,x in user_upvotes.most_common(10):
    print("{}: {}".format(t, x))

In [ ]:
#Ausgabe der Ergebnisse in Grafiken

#LDA Model Visualisiert
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model, allposts, allwords)
pyLDAvis.display(panel)

In [ ]:
#Wordclouds der ermittelten Topics

#Erzeugt eine figure in der die wordclouds einen eigenen Subplot bekommen, Wordclouds überschreiben sich sonst
fig, axes = pyplot.subplots(1, len(lda_model.print_topics()), figsize=(5*len(lda_model.print_topics()), 5))

for i in lda_model.print_topics():
    wordcloud = WordCloud(background_color="white", width=800, height=800, colormap="cividis")
    wordcloud.generate(i[1])
    axes[i[0]].imshow(wordcloud)
    axes[i[0]].axis('off')

pyplot.show()

In [ ]:
#Wordcloud der häufigsten Wörter
wordcloud = WordCloud(background_color="white", max_words=50, width=800, height=800, colormap="twilight_shifted")
wordcloud.generate(end_string)
wordcloud.to_image()

In [ ]:
#Diagramme der aktivsten und meist hochghewählten Nutzer

#Sortiere die Nutzer nach Anzahl an Beiträgen
top_contributors = sorted(user_contributionscounter.items(), key=lambda item: item[1], reverse=True)[:10]

#Extrahiere Namen und Anzahl an Beiträgen der Top 10 Redditor
contributor = [str(c[0].name) for c in top_contributors]
contributions = [c[1] for c in top_contributors]

#Erstelle Diagramm
pyplot.bar(contributor, contributions)
pyplot.xlabel("Redditor")
pyplot.ylabel("Anzahl an Beiträgen")
pyplot.title("Top 10 aktivste Redditor")
pyplot.xticks(rotation=90)
pyplot.show()


#Sortiere die Nutzer nach Anzahl an Upvotes
top_contributors = sorted(user_upvotes.items(), key=lambda item: item[1], reverse=True)[:10]

#Extrahiere Namen und Anzahl an Upvotes der Top 10 Redditor
contributor = [str(c[0].name) for c in top_contributors]
upvotes = [c[1] for c in top_contributors]

#Erstelle Diagramm
pyplot.bar(contributor, upvotes, color ='maroon')
pyplot.xlabel("Redditor")
pyplot.ylabel("Anzahl an Upvotes")
pyplot.title("Top 10 meist hochgewählte Redditor")
pyplot.xticks(rotation=90)
pyplot.show()